In [20]:
import tensorflow as tf
import numpy as np
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.3)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

## Tensorflow变量

### tf.one_hot

In [2]:
z = np.random.randint(0, 10, size=[10])
y = tf.one_hot(z, 10, on_value=1, off_value=None, axis=0)
with tf.Session() as sess:
    print(z, '\n')
    print(sess.run(y))

[1 1 0 2 6 6 1 5 1 3] 

[[0 0 1 0 0 0 0 0 0 0]
 [1 1 0 0 0 0 1 0 1 0]
 [0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 1 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]


### tf.get_variable 和 tf.variable  
1.使用tf.Variable时，如果检测到命名冲突，系统会自己处理。使用tf.get_variable()时，系统不会处理冲突，而会报错  
2.由于tf.Variable() 每次都在创建新对象，所有reuse=True 和它并没有什么关系。对于get_variable()，来说，如果已经创建的变量对象，就把那个对象返回，如果没有创建变量对象的话，就创建一个新的。

In [8]:
w_1 = tf.Variable(3, name='W_1')
w_2 = tf.Variable(1, name='W_1')
print(w_1.name)
print(w_2.name)

W_1_4:0
W_1_5:0


In [9]:
w_1 = tf.get_variable(name='W_1', initializer=1)
w_2 = tf.get_variable(name='W_1', initializer=1)

ValueError: Variable W_1 already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "<ipython-input-9-a8ab65406ab8>", line 1, in <module>
    w_1 = tf.get_variable(name='W_1', initializer=1)
  File "/home/lhw/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2862, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/home/lhw/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2802, in run_ast_nodes
    if self.run_code(code, result):


In [10]:
with tf.variable_scope('scope1'):
    w1 = tf.get_variable('W1', shape=[])
    w2 = tf.Variable(0.0, name='W2')
with tf.variable_scope('scope2'):
    w1_p = tf.get_variable('W1', shape=[])
    w2_p = tf.Variable(1.0, name='W2')
print(w1 is w1_p, w2 is w2_p)

False False


In [4]:
#用tf.get_variable来定义变量
def conv_relu(input, kernel_shape, bias_shape):
    weight = tf.get_variable("weight", kernel_shape, initializer=tf.random_normal_initializer())
    biases = tf.get_variable("biases", bias_shape, initializer=tf.constant_initializer(0.0))
    conv = tf.nn.conv2d(input, weight, strides=[1, 1, 1, 1], padding='SAME')
    return tf.nn.relu(conv + biases)

In [7]:
#用tf.variable_scope来区分变量的范围
def my_image_filter(input_images):
    with tf.variable_scope("conv1"):
        # Variables created here will be named "conv1/weights", "conv1/biases"
        relu1 = conv_relu(input_images, [5, 5, 32, 32], [32])
    with tf.variable_scope("conv2"):
        # Variables created here will be named "conv2/weights", "conv2/biases".
        return conv_relu(relu1, [5, 5, 32, 32], [32])

In [ ]:
#scope.reuse_variables()来对不同的图片共享变量
with tf.variable_scope("image_filters") as scope:
    result1 = my_image_filter(image1)
    scope.reuse_variables()
    result2 = my_image_filter(image2)

## TensorFlow损失函数

### 分类问题损失函数——交叉熵（crossentropy）

In [11]:
y_ = tf.constant([[1.0, 0, 0]]) #一对括号表示的是列表，两对括号才表示矩阵  即表示1*3 正确的标签
# 以下为未经过Softmax处理的类别得分  
y3 = tf.constant([[10.0, 3.0, 2.0]])#预测结果1
y4 = tf.constant([[5.0, 3.0, 1.0]])#预测结果2
# TensorFlow提供的集成交叉熵  
# 注：该操作应该施加在未经过Softmax处理的logits上，否则会产生错误结果  
# labels为期望输出，且必须采用labels=y_, logits=y的形式将参数传入
cross_entropy_v2_1 = tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y3)
cross_entropy_v2_2 = tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y4)
sess = tf.InteractiveSession()
print('v2_1', cross_entropy_v2_1.eval())
print('v2_2', cross_entropy_v2_2.eval())
sess.close()

v2_1 [ 0.00124651]
v2_2 [ 0.1429317]


### 回归问题损失函数——均方误差（MSE，mean squared error）

### 自定义损失函数

## tf.placeholder函数

In [3]:
x = tf.placeholder(tf.float32, shape=(1024, 1024))
y = tf.matmul(x, x)

with tf.Session() as sess:
    #print(sess.run(y))
    rand_array = np.random.rand(1024, 1024)
    print(sess.run(y, feed_dict={x: rand_array}))
    sess.close()

[[ 245.76368713  253.77127075  247.28686523 ...,  253.14624023
   257.20074463  257.65789795]
 [ 258.61959839  266.53594971  254.59066772 ...,  262.24328613
   265.61923218  275.69683838]
 [ 257.12890625  264.57775879  258.34194946 ...,  255.92092896
   262.13800049  265.87683105]
 ..., 
 [ 258.13299561  260.77801514  251.29177856 ...,  260.17715454
   260.57437134  266.06335449]
 [ 250.81408691  260.76593018  251.63342285 ...,  261.22869873
   262.38302612  268.05218506]
 [ 254.83059692  265.75665283  258.72134399 ...,  263.22998047
   261.13891602  270.49310303]]


In [4]:
input1 = tf.placeholder(tf.float32)
input2 = tf.placeholder(tf.float32)

output = tf.multiply(input1, input2)

with tf.Session() as sess:
    print(sess.run(output, feed_dict = {input1:[3.], input2:[4.]}))

[ 12.]


## tensorflow-BatchNormalization

In [21]:
W = tf.constant([[-2., 12., 6.], [3., 2., 8.]])
mean, var = tf.nn.moments(W, axes=[0])

In [22]:
sess.run(tf.global_variables_initializer())
resultMean = sess.run(mean)
print(resultMean)
resultVar = sess.run(var)
print(resultVar)

[ 0.5  7.   7. ]
[  6.25  25.     1.  ]


In [24]:
size = 3
scale = tf.Variable(tf.ones([size]))
shift = tf.Variable(tf.zeros([size]))
epislon = 0.001
W = tf.nn.batch_normalization(W, mean, var, shift, scale, epislon)

In [25]:
sess.run(tf.global_variables_initializer())
resultW = sess.run(W)
print(resultW)

[[-0.99992001  0.99997997 -0.99950027]
 [ 0.99991995 -0.99997997  0.99950027]]


## tf.reduce_mean()和tf.reduce_sum()  
降维求  
求每个维度的和最大值tf.reduce_max(input_tensor, reduction_indices=None, keep_dims=False, name=None)  
求每个维度的平均值tf.reduce_mean(input_tensor, reduction_indices=None, keep_dims=False, name=None)
求每个维度的和tf.reduce_sum(input_tensor, reduction_indices=None, keep_dims=False, name=None)

In [30]:
X = tf.constant([[1.0, 1.0, 1.0], [1.0, 1.0, 1.0],[1.0, 1.0, 1.0]])
print(tf.reduce_sum(X))
print(tf.reduce_sum(X, 0))
print(tf.reduce_sum(X, 1))
print(tf.reduce_sum(X, 1, keep_dims=True))
print(tf.reduce_mean(X, [0, 1]))

Tensor("Sum_20:0", shape=(), dtype=float32)
Tensor("Sum_21:0", shape=(3,), dtype=float32)
Tensor("Sum_22:0", shape=(3,), dtype=float32)
Tensor("Sum_23:0", shape=(3, 1), dtype=float32)
Tensor("Mean_1:0", shape=(), dtype=float32)
